# 1.何为UniMP?
**原文镇楼**：[Masked Label Prediction: Unified Message Passing Model for Semi-Supervised Classification](https://arxiv.org/pdf/2009.03509.pdf)

一般应用于半监督节点分类的算法分为**图神经网络**和**标签传递**算法两类，它们都是通过消息传递的方式（前者传递特征、后者传递标签）进行节点标签的学习和预测。百度PGL团队提出的统一消息传递模型UniMP，将上述两种消息统一到框架中，同时实现了节点的特征与标签传递，显著提升了模型的泛化效果。

在一般的分类问题中，一个样本的标签通常与其他样本的标签毫无关联，因此预测仅仅依赖于每个样本各自的特征。而在图网络中，一个节点的标签可能会影响到其它节点的标签，原因在于通过边直接或间接相连的两个节点存在着某种联系。因此标签传递的引入有利于充分利用已知节点的信息和网络连接信息，提升预测精度。

![](https://ai-studio-static-online.cdn.bcebos.com/c9be38f9dc8b466985fe43ad9dd4d56f11a3103d76cd493d9c2f0ac627f44c68)

# 2.为何引入MaskLabel?
简单的加入标签信息会带来**标签泄漏**的问题，即标签信息即是特征又是训练目标。可以想象直接将标签作为网络输入，要求输出也向标签靠拢，势必会造成“1=1”的训练结果，无法用于预测。

在引用网络中，论文是按照时间先后顺序出现的，其标签也应该有一定的先后顺序。在无法得知训练集标签顺序的情况下，UniMP提出了MaskLabel学习方法。每一次随机将一定量的节点标签掩码为未知，用部分已有的标注信息、图结构信息以及节点特征来还原训练数据的标签。

```
def label_embed_input(self, feature):
	label = F.data(name="label", shape=[None, 1], dtype="int64")
	label_idx = F.data(name='label_idx', shape=[None, 1], dtype="int64")
	label = L.reshape(label, shape=[-1])
	label_idx = L.reshape(label_idx, shape=[-1])

	embed_attr = F.ParamAttr(initializer=F.initializer.NormalInitializer(loc=0.0, scale=1.0))
	embed = F.embedding(input=label, size=(self.out_size, self.embed_size), param_attr=embed_attr)
	feature_label = L.gather(feature, label_idx, overwrite=False)
	feature_label = feature_label + embed
	feature = L.scatter(feature, label_idx, feature_label, overwrite=True)
        
	lay_norm_attr = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=1))
	lay_norm_bias = F.ParamAttr(initializer=F.initializer.ConstantInitializer(value=0))
	feature = L.layer_norm(feature, name='layer_norm_feature_input', param_attr=lay_norm_attr, bias_attr=lay_norm_bias)
    
	return feature
```

在上面的代码中可以看到，对于已知标签的节点，首先将其embedding成和节点特征同样维度（这里是100维），然后就可以直接与节点特征相加，进而完成了标签信息与特征信息的融合，一块送入graph_transformer进行消息传递。

**改进**：这里，最核心的一句代码是feature_label = feature_label + embed，它完成了标签和特征的融合，由此可以想到控制两者的权重，得到：
```
feature_label = alpha*feature_label + (1-alpha)*embed
```
alpha可以设定为固定值，也可以通过学习获得。参考model_unimp_large.py中的门控残差连接：
```
if gate:
	temp_output = L.concat([skip_feature, out_feat, out_feat - skip_feature], axis=-1)
	gate_f = L.sigmoid(linear(temp_output, 1, name=name + '_gate_weight', init_type='lin'))
	out_feat = skip_feature * gate_f + out_feat * (1 - gate_f)
else:
	out_feat = skip_feature + out_feat
```
可以写出：
```
temp = L.concat([feature_label,embed,feature_label-embed], axis=-1)
alpha = L.sigmoid(linear(temp, 1, name='alpha_weight', init_type='lin'))
feature_label = alpha*feature_label + (1-alpha)*embed
```
当然也可以直接经过一层FC后再将两者相加：
```
feature_label = L.fc(feature_label, size=100) + L.fc(embed, size=100)
```
而做这些的目的，都是为了寻找能使标签信息和特征信息融合的更好的方式。

# 3.Res连接&Dense连接?
（1）残差网络（或称深度残差网络、深度残差学习，英文ResNet）属于一种卷积神经网络。相较于普通的卷积神经网络，残差网络采用了跨层恒等连接，以减轻卷积神经网络的训练难度。残差网络的一种基本模块如下所示：
![](https://ai-studio-static-online.cdn.bcebos.com/1897b27026a6462aa1498bab691fca9ef591e3f7754c40369608b08231bcf3ba)

实现起来比较简单，这里不予赘述。

（2）DenseNet原文：[Densely Connected Convolutional Networks](https://arxiv.org/pdf/1608.06993.pdf)

相比ResNet，DenseNet提出了一个更激进的密集连接机制：即互相连接所有的层，具体来说就是每个层都会接受其前面所有层作为其额外的输入。DenseNet的网络结构如下所示：
![](https://ai-studio-static-online.cdn.bcebos.com/626c1616e637450b97b4150bfefd04a1dcefd6341f01401996046026955ae82a)

以下代码实现了Dense连接：
```
dense=[feature]
for i in range(self.num_layers - 1):
	ngw = pgl.sample.edge_drop(graph_wrapper, edge_dropout) 
	res_feature = feature

	feature, _, cks = graph_transformer(str(i), ngw, feature,
	hidden_size=self.hidden_size,
	num_heads=self.heads, 
		concat=True, skip_feat=True,
 		layer_norm=True, relu=True, gate=True)
	if dropout > 0:
	feature = L.dropout(feature, dropout_prob=dropout, dropout_implementation='upscale_in_train') 

	dense.append(feature)
	feature = L.fc(dense, size=self.hidden_size, name="concat_feature")
```

# 3.注意力机制?
注意力机制就是将注意力集中于局部关键信息的机制，可以分成两步：第一，通过全局扫描，发现局部有用信息；第二，增强有用信息并抑制冗余信息。SENet是一种非常经典的注意力机制下的深度学习方法。它可以通过一个小型的子网络，自动学习得到一组权重，对特征图的各个通道进行加权。其含义在于，某些特征通道是较为重要的，而另一些特征通道是信息冗余的；那么，我们就可以通过这种方式增强有用特征通道、削弱冗余特征通道。SENet的一种基本模块如下所示：
![](https://ai-studio-static-online.cdn.bcebos.com/7c019859e3ec427c84a05a319503db11fb0e350e5b6740b3930f0c613ce7f6e0)

值得指出的是，通过这种方式，每个样本都可以有自己独特的一组权重，可以根据样本自身的特点，进行独特的特征通道加权调整。

Unimp中的注意力机制出现在Graph Transformer以及最后的输出层attn_appnp，attn_appnp的代码为：
```
def attn_appnp(gw, feature, attn, alpha=0.2, k_hop=10):
	"""Attention based APPNP to Make model output deeper
	Args:
		gw: Graph wrapper object (:code:`StaticGraphWrapper` or :code:`GraphWrapper`)
		attn: Using the attntion as transition matrix for APPNP
		feature: A tensor with shape (num_nodes, feature_size).
		k_hop: K Steps for Propagation
	Return:
		A tensor with shape (num_nodes, hidden_size)
	"""
	def send_src_copy(src_feat, dst_feat, edge_feat):
		feature = src_feat["h"]
		return feature

	h0 = feature
	attn = L.reduce_mean(attn, 1)
	for i in range(k_hop):
		msg = gw.send(send_src_copy, nfeat_list=[("h", feature)])
		msg = msg * attn
		feature = gw.recv(msg, "sum")
		feature = feature * (1 - alpha) + h0 * alpha
	return feature
```
在调用函数时，其中的alpha为前面的graph_transformer学习到的参数，用于更好的融合各层特征。
# 5.结语
在调通UniMP之后，后续尝试的技巧对于其精度的提升效力微乎其微，所以不得不再次感叹百度PGL团队的强大！

再一次附上UniMP的GitHub链接：[戳我](https://github.com/PaddlePaddle/PGL/tree/main/ogb_examples/nodeproppred/unimp)

由于笔者才疏学浅，文中难免存在一些错误或疏漏，恳请各位专家学者批评指正！

# 参考文献
【1】[《注意力机制+软阈值化=深度残差收缩网络（深度学习）》](https://www.cnblogs.com/davidxiong2020/p/12447061.html)

【2】[DenseNet详解](https://zhuanlan.zhihu.com/p/43057737)

【3】[百度飞桨登顶图神经网络权威榜单3项榜首 推出大杀器UniMP ](https://www.sohu.com/a/419637951_120630247)